In [170]:
# to start the playwright session:
# 
# # PC
## "C:\Program Files\Google\Chrome\Application\chrome.exe" --remote-debugging-port=9223 --user-data-dir="C:\my-chrome-profile"

# mac
# /Applications/Google\ Chrome.app/Contents/MacOS/Google\ Chrome --remote-debugging-port=9223 --user-data-dir="~/chrome-debug-profile"


###################################

# to start the server which will communicate with the playwright session:
## C:/Users/dusan/OneDrive/Desktop/pp/test/crawl/.venv/Scripts/python.exe fastAPIServ.py


## git add .
## git commit -m "message"
## git push -u origin main

In [6]:

%load_ext autoreload
%autoreload 2

import helper as hlp
import get_2fa as g2

import hlogger as hlog
import os
from log_config import get_logger

logger = get_logger("Jupyter Notebook")  # Use module name for easier identification

2025-02-02 15:10:55,824 - helper - <module> - Line: 916 - INFO - Running on Windows
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:



client = 'intuit'
wd = os.getcwd()  # Gets the current working directory
client_folder = 'clients/'+client
# --- Configuration ---
site_description = "Quickbooks Online Website"
site_wide_instructions = """
RULES:
1. As you navigate the site, attempt to click the expansion menu link options and drop downs instead of the main menu option to ensure that the most appropriate link is selected. For example if you see a navigational item called 'Payroll' and it has an expansion button next to it, click the expansion button first then make a more informed selection. Same with other buttons with expansion options. The expansion may offer a comprehensive view, but also it may offer an alternate view where the actual correct choice is simply the top level item. Either way, it is importnat to be thorough.
2. When completing forms some elements require drop down selections. Make sure to click on these first before entering text into the field, as text entry may not be accepted if the drop down is not selected first.

"""
workflow_id = "add_non_inventory_product2" #unique identifier for the workflow
workflow_instructions = """Add a new non-inventory type product by navigating to sales->products->New->non-inventory type. Fill out the form with mock data and save the form.
Non-inventory type product is a products you buy and/or sell but don’t need to (or can’t) track quantities of, for example, nuts and bolts used in an installation.
""" #instructions for the workflow
# workflow_instructions = "Add a new customer manually. Follow the icons at the top of the Customer entry form to enter different sections of customer info before saving the form. Enter just basic example data." #instructions for the workflow
#workflow_instructions = "Add sleep mask to cart then go to cart and remove it" #instructions for the workflow

client_url = 'https://sandbox.qbo.intuit.com/app/homepage'
hlp.navigate_to_url_from_metadata(client_url)

extra_instructions = ""
logger_struct = hlog.HierarchicalLogger(client_folder = client_folder)
logger_struct.set_workflow(workflow_id)
logger_struct.set_workflow_value("site_wide_instructions", site_wide_instructions)
logger_struct.set_workflow_value("site_description", site_description)
logger_struct.set_workflow_value("workflow_instructions", workflow_instructions)

logger.info(f"Starting workflow {workflow_id} for {client}")



2025-02-02 15:18:42,985 - Jupyter Notebook - <module> - Line: 29 - INFO - Starting workflow add_non_inventory_product2 for intuit


In [10]:
import math
import requests

def fold_screenshot_test(starting_width,starting_height,depth_chunks_to_test,run_rerun_path,workflow_instructions,site_wide_instructions,site_description,directory):

    #run_rerun_path = 'clients/qb\\cust3\\0\\0\\0\\0'
    logger.info(f"Requesting extract_metadata for {client} with starting width {starting_width} and starting height {starting_height}")
    # scans through the page and extracts metadata
    metadata_response = requests.get(f"http://127.0.0.1:8000/extract_metadata?steps={10}&wait_per_step_ms={1000}&overlap_percent=30",)
    if metadata_response.status_code != 200:
        logger.error(f"Failed to fetch metadata. Status code: {metadata_response.status_code}. Message: {metadata_response}")
        raise Exception(f"Failed to fetch metadata. Status code: {metadata_response.status_code}. Message: {metadata_response}")
        
    url_metadata = metadata_response.json()
    url_metadata



    # modified_height,tokens = hlp.largest_height_given_width(max_tokens=1600,orig_width=starting_width,suggested_height=starting_height)

    import copy
    new_height = copy.deepcopy(starting_width)

    full_height = url_metadata['dimensions']['fullPage']['height']

    diff = full_height - new_height
    increment = math.ceil(diff/depth_chunks_to_test)


    fold_tests = []

    i = 1
    while i <= depth_chunks_to_test:
        logger.info(f"Resizing browser to width {starting_width} and height {new_height}")
        resize_browser_result = hlp.resize_browser_window(width=starting_width, height=new_height)
        fold_tests.append({'width':starting_width,'height':new_height,'id':i})

        logger.info(f"Requesting screenshot for chunk {i}")
        screenshot_response = requests.get(
            f"http://127.0.0.1:8000/screenshot",## todo
            params={"output_path": run_rerun_path+f"/{directory}", "overlap_percentage": 0,"max_chunks": 1, 
                        "action_id": None,
                        "candidate_id": None,
                        "single_chunk_override_id":i
                    }
        )
        logger.info(f"Completed screenshot for chunk {i}")
        new_height = new_height + increment
        i += 1
    resize_browser_result = hlp.resize_browser_window(width=starting_width, height=starting_height)

    fold_test_paths = []
    #chunk_file_list =  hlp.get_files_with_extension(run_rerun_path+"/chunks", ".png")
    for c in fold_tests:
        logger.info(f"Resizing screenshot {c['id']}")
        tmp_resize_screenshot_result = hlp.resize_and_crop(
            input_path=f'{run_rerun_path}/{directory}/{f'chunk_{c['id']}.png'}',
            output_path=f'{run_rerun_path}/{directory}/resized_{f'chunk_{c['id']}.png'}',
        )
        c['path'] = run_rerun_path+f"/{directory}/resized_chunk_{c['id']}.png"
        c['file_name'] = f"resized_chunk_{c['id']}.png"
        fold_test_paths.append(c['path'])
        tmp_resize_screenshot_result['id'] = c['id']
        c['new_dimensions'] = {'height':tmp_resize_screenshot_result['new_height'],'width':tmp_resize_screenshot_result['new_width']}
        #print(c['id'],":",'width:',c['new_dimensions'])

###################


    return fold_tests,fold_test_paths,screenshot_response,url_metadata,resize_browser_result

def perform_fold_test_llm(fold_tests,fold_test_paths,screenshot_response,resize_browser_result):

    page_action_json_ranked_fold_test = None

    if len(fold_tests) > 1:
        logger.info(f"Requesting fold test llm process because there are {len(fold_tests)} fold tests")

        page_action_json_ranked_fold_test,action_prompt,raw = hlp.perform_llm_fold_test(fold_tests,fold_test_paths,workflow_instructions,site_wide_instructions,site_description)

        best_image={}
        best_image = max(
            page_action_json_ranked_fold_test,
            key=lambda x: (x['relevant_elements'], x['clarity']),
        )
        for f in fold_tests:
            if f['id'] == best_image['id']:
                best_image['path'] = f['path']
                best_image['file_name'] = f['file_name']
                best_image['resized_dimensions'] = f['new_dimensions']
                best_image['original_dimensions'] = {'height':f['height'],'width':f['width']}
        
        logger.info(f"Resizing window for best image is {best_image['file_name']} with clarity {best_image['clarity']} and {best_image['relevant_elements']} relevant elements")
        
        resize_browser_result = hlp.resize_browser_window(width=best_image['original_dimensions']['width'], height=best_image['original_dimensions']['height'])

    else:
        best_image={}
        best_image['id'] = fold_tests[0]['id']
        best_image['path'] = fold_tests[0]['path']
        best_image['file_name'] = fold_tests[0]['file_name']
        best_image['resized_dimensions'] = fold_tests[0]['new_dimensions']
        best_image['original_dimensions'] = {'height':fold_tests[0]['height'],'width':fold_tests[0]['width']}

    #best_image

    # resize browse to the best fold

    screenshot_data = screenshot_response.json()

    return page_action_json_ranked_fold_test,best_image,screenshot_data,resize_browser_result



In [11]:


import time
import json
import copy
import requests

max_run = 11
max_rerun = 4
max_run_rerun = 4

max_chunks = 10
overlap_percentage = 0
draw_dots = True


run_id = 0
rerun_ct = 0
run_rerun_ct = 0
advice = ''
summary = ''
goal_achieved = False
any_actions_succeeded = False
resize_data = None  # Store resize data between runs

# Main loop

actions_succeeded = False
page_changed = False
overall_goal_success = False

stability_timeout_ms = 10000
stability_window_ms = 1000
check_stability_x_before_error = 5

wait_after_scroll = 500


# 1) Set context
sample_id = logger_struct.set_sample()
logger_struct.set_rerun(rerun_ct)
logger_struct.set_run(run_id)
logger_struct.set_run_retry(run_rerun_ct)

# todo: 
# test what happens when a rerun is needed
# add "generalized instructions" to the actions
import math
import requests

starting_width = 1256
starting_height=1369
depth_chunks_to_test = 2
run_image_double_check = False

# dusan_tracker['goal']=workflow_instructions

#resize_browser_result = hlp.resize_browser_window(width=browser_start_width, height=browser_start_height)
run_rerun_path = ''


while run_id <= max_run and rerun_ct <= max_rerun and not overall_goal_success and run_rerun_ct <= max_run_rerun:

    
    #print(logger.context)
    # deep copy run_rerun_path
    previous_run_rerun_path = copy.deepcopy(run_rerun_path)
    #print(f"Now previous = {previous_run_rerun_path} which is run {run_rerun_path}")
    run_rerun_path = os.path.join(client_folder, workflow_id,str(sample_id),str(rerun_ct),str(run_id),str(run_rerun_ct))
    logger.info(f"Previous run_rerun_path is {previous_run_rerun_path} and current run_rerun_path is {run_rerun_path}")
    #print(f"And Run is now: {run_rerun_path}")
    logger.info(f"Starting run {run_id} rerun {rerun_ct} run_rerun {run_rerun_ct} in {run_rerun_path}")


    start_time = time.time()

    # Set up folders
    
    if run_id == 0:
        summary = ''



        ############################################


        logger.info(f"It's the first run for the sample. Starting fold screenshot test")
        fold_tests,fold_test_paths,screenshot_response,resize_browser_result,url_metadata = fold_screenshot_test(starting_width=starting_width,starting_height=starting_height,depth_chunks_to_test=depth_chunks_to_test,run_rerun_path=run_rerun_path,workflow_instructions=workflow_instructions,site_wide_instructions=site_wide_instructions,site_description=site_description,directory='fold_test')

        logger.info(f"Completed fold screenshot test. Starting fold test LLM analysis for {len(fold_tests)} fold tests")

        page_action_json_ranked_fold_test,best_image,screenshot_data,resize_browser_result = perform_fold_test_llm(fold_tests,fold_test_paths,screenshot_response,resize_browser_result)

        ############################################



        # First run in a rerun sequence OR stalled - get fresh screenshot and metadata
        # NOTICE: this needs to be done only once per rerun sequence (unless the browser is messed with durnin the run)
        
        # url_metadata, screenshot_data = hlp.run_metadata_gather(
        #     output_path=run_rerun_path+"/chunks",
        #     overlap_percentage=overlap_percentage,
        #     max_chunks=max_chunks
        # )

        if rerun_ct == 0:
            logger.info(f"First run in the rerun sequence. Deleting advice")
            advice = ''

        resize_data = {
            'resize_browser_result': resize_browser_result,
            #'resize_screenshot_result': resize_screenshot_result,
            'best_image':best_image,
            'url_metadata': url_metadata,
            'screenshot_data': screenshot_data
        }

        logger.info(f"Copying screenshot from fold test folder to chunk folder for best image for resized and non rezised for best image id {best_image['id']}")

        hlp.copy_and_rename_file(f'{best_image['path']}', f'{run_rerun_path}/chunks/resized_chunk_{best_image['id']}.png',delete_original=False)
        hlp.copy_and_rename_file(f'{run_rerun_path}/fold_test/chunk_{best_image['id']}.png', f'{run_rerun_path}/chunks/chunk_{best_image['id']}.png',delete_original=False)

    else:
        logger.info(f"Subsequent run in the same rerun sequence. Reusing data from previous run. Advice length is {len(advice)} characters")
        # Subsequent runs - copy screenshot from temp and reuse resize data


        try: # we write to temp so that we can compare once we have the new screenshot

            # temp was captured for eval in the 0th and all other runs

            logger.info(f"Copying screenshot from temp to new run folder: {previous_run_rerun_path}/temp to {run_rerun_path}/chunks")

            if len(previous_run_rerun_path) == 0:
                logger.error(f"Copying screenshot to new run folder: {previous_run_rerun_path} is empty")
            temp_file_list =  hlp.get_files_with_extension(previous_run_rerun_path+"/temp", ".png")
            for tf in temp_file_list:
                hlp.copy_and_rename_file(
                    f'{previous_run_rerun_path}/temp/{tf}',
                    f'{run_rerun_path}/chunks/{tf}',
                    delete_original=False
                )

            # Reuse resize data from first run
            resize_browser_result = resize_data['resize_browser_result']
            #resize_screenshot_result = resize_data['resize_screenshot_result']
            best_image = resize_data['best_image']
            url_metadata = resize_data['url_metadata']
            screenshot_data = resize_data['screenshot_data']
            
            # hlp.add_step_to_run_data(run_data, 'reused_data', 'Using copied data from previous run', {
            #     "url_metadata": url_metadata,
            #     "screenshot_data": screenshot_data,
            #     "resize_screenshot_result": resize_screenshot_result
            # })
            
        except Exception as e:
            logger.error(f"Error copying screenshot to new run folder: {e}")
            continue




    logger.info(f"Starting LLM analysis with advice length {len(advice)} characters")

    page_action_json_ranked,action_prompt = hlp.perform_llm_analysis(
        extra_instructions,
        site_wide_instructions,site_description,
        run_rerun_path,
        workflow_instructions,
        #resize_screenshot_result, # TODO: change to be broader
        best_image,
        run_id,
        start_time
        ,advice
        ,summary
    )
    
    logger.info(f"Completed LLM analysis. {len(page_action_json_ranked['action_tasks'])} actions to perform")



    if page_action_json_ranked is None:
        logger.error("LLM analysis failed to produce valid results")
        continue
    else:
        if len(page_action_json_ranked['action_tasks']) == 0:
            logger.error("LLM analysis failed to produce valid results")
            continue
    #hlp.add_step_to_run_data(run_data, 'llm_analysis', 'LLM analysis for action planning', page_action_json_ranked)

    # print("\nDrawing elements and creating highlighted screenshot...")
    # print(f"Page action JSON: {json.dumps(page_action_json_ranked, indent=2)}")


    logger_struct.set_run_retry_value("summary_of_steps_so_far", page_action_json_ranked['summary_of_steps_so_far'])
    logger_struct.set_run_retry_value("expected_outcome_hopeful", page_action_json_ranked['expected_outcome_hopeful'])
    logger_struct.set_run_retry_value("_advice_assessment", page_action_json_ranked['_advice_assessment'])
    logger_struct.set_run_retry_value("page_description", page_action_json_ranked['page_description'])


    

## left off
    
    # Prepare actions to perform
    actions_to_perform = []
    for task in page_action_json_ranked.get('action_tasks', []):
        logger_struct.set_action(task['action_id'])
        logger_struct.set_action_value("description", task['description'])
        # Fix the string concatenation by ensuring proper order of operations
        # dusan_tracker.append("    -Action tasks: ID:" + str(task['action_id']) + " Descript: " + str(task['description']))

        for c in task['candidates']:
            logger_struct.set_candidate(c['candidate_id'])
            logger_struct.set_candidate_value(key="element_description", value=c['element_description'])
            logger_struct.set_candidate_value(key="type_text", value=c.get('type_text', ''))
            logger_struct.set_candidate_value(key="keyboard_action", value=c.get('keyboard_action', ''))
            logger_struct.set_candidate_value(key="image_number", value=c.get('image_number', ''))
            logger_struct.set_candidate_value(key="action", value=c['action'])
            logger_struct.set_candidate_value(key="to_act", value=c['to_act'])

        action_candidates = {
            'description': task['description'],
            'action_id': task['action_id'],
            'candidates': [c for c in task['candidates'] if c.get('to_act', False)]
        }

        actions_to_perform.append(action_candidates)


    # Prepare the action request payload
    action_request_payload = {
        'multiple_steps_required': page_action_json_ranked.get('multiple_steps_required', False),
        'visible_elements_from_instructions': page_action_json_ranked.get('visible_elements_from_instructions', ''),
        'summary_of_steps_so_far': page_action_json_ranked.get('summary_of_steps_so_far', ''),
        'action_tasks': actions_to_perform,
        'error': page_action_json_ranked.get('error', None),
        'page_description': page_action_json_ranked.get('page_description', ''),
        'expected_outcome_hopeful': page_action_json_ranked.get('expected_outcome_hopeful', '')
    }


    # Execute actions
    #response = hlp.execute_actions(action_request_payload, 2)  # The 2 is the wait_time parameter

    # Execute actions using the API server (FastAPI).
    logger.info(f"Sending screenshotting request to API server for {len(action_request_payload['action_tasks'])} actions")
    draw_response = hlp.send_action_request(action_request_payload, wait_time=2,
        run_rerun_path=run_rerun_path,draw_no_action=True)
    

    if run_image_double_check:
        logger.info(f"Double checking the image draw dots")
        coordinate_info_to_review = []
        for a in action_request_payload['action_tasks']:
            coordinate_info_to_review.append({'action_id':a['action_id'],'description':a['description'],'element_description':a['candidates'][0]['element_description'],'coordinates':a['candidates'][0]['coordinates_ready_to_act'],'action':a['candidates'][0]['action']})

        coor_check_results,coor_check_prompt,coor_check_raw = hlp.check_coordinates_llm(workflow_instructions,site_wide_instructions,site_description,run_rerun_path,coordinate_info_to_review)
    else:
        logger.info(f"Skipping double checking the image draw dots")
    

    logger.info(f"Sending action request to API server for {len(action_request_payload['action_tasks'])} actions")
    action_response = hlp.send_action_request(action_request_payload, wait_time=2,
        run_rerun_path=run_rerun_path,draw_no_action=False)

    # not shure if this is needed
    hlp.resize_and_crop(
            f'{run_rerun_path}/dots/final_screenshot.png',
            f'{run_rerun_path}/dots/resized_final_screenshot.png'
        )



    actions_succeeded = False
    page_changed = False
    goal_achieved = False
    overall_goal_success = False

    if action_response:
        logger.info(f"Executed run actions for {len(action_response)} actions")


        if type(action_response[0]['element_metadata'])!=dict:
            logger.error("Element metadata is not a dictionary")
        elif len(action_response[0]['element_metadata']) == 0:
            logger.error("No element metadata in the response")
        else:
            logger.info(f"Found {len(action_response[0]['element_metadata'])} out of {len(action_response)} element metadata in the response")

        highlighting_metadata = {}
        for at in page_action_json_ranked['action_tasks']:
            #print(at)
            for c in at['candidates']:
                #print(c)
                for r in action_response:
                    if r['action_id'] == at['action_id'] and r['candidate_id'] == c['candidate_id']:
                        # dusan_tracker.append("         -Acted on Action tasks: aID:" + str(r['action_id']) + " cID:" + str(r['candidate_id']))

                        


                        if c.get('action','') != 'keyboard_action' and (c.get("keyboard_action", '') is None or c.get("keyboard_action", '').strip()==''):
                            c['element_metadata'] = r['element_metadata']
                            new_box = copy.deepcopy(c['element_metadata']['boundingBox'])
                            #new_box['y'] = new_box['y'] - c['scroll_to']

                            if at['action_id'] in highlighting_metadata:
                                highlighting_metadata[at['action_id']].append(new_box)
                                
                                    
                            else:
                                highlighting_metadata[at['action_id']] = [new_box]
                        else:
                            c['element_metadata'] = {}
        
        logger.info(f"Creating highlighted screenshot with {len(highlighting_metadata)} boxes")

        file_for_cairo = next((s for s in hlp.get_files_with_extension(run_rerun_path+"/chunks", ".png") if s.startswith("chunk")), None)
        hlp.create_highlighted_screenshot_cairo(
            run_rerun_path=run_rerun_path, # Use the current
            bbox_payload = highlighting_metadata
            ,file_for_cairo = file_for_cairo
        )
        # check for stability before proceeding by checking if the page is stable

        # todo: create stability check
        # sleep for 5 seconds
        time.sleep(2)       
        # stability = False
        # for i in range(check_stability_x_before_error):

        #     screenshot_response = requests.get(
        #         f"http://127.0.0.1:8000/check-stability?timeout_ms={stability_timeout_ms}&window_size_ms={stability_window_ms}",
        #     )
        #     if screenshot_response.json()['is_stable']:
        #         stability = True
        #         break
        # if not stability:
        #     print("Error!!!!!!!!!!!!!!!!!!!: Stability not reached")
        #     break

########################################################
   

        logger.info(f"Requesting extract_metadata with starting width {starting_width} and starting height {starting_height}")

        fold_tests_temp,fold_test_paths_temp,screenshot_response_temp,resize_browser_result_temp,url_metadata_temp = fold_screenshot_test(starting_width=starting_width,starting_height=starting_height,depth_chunks_to_test=depth_chunks_to_test,run_rerun_path=run_rerun_path,workflow_instructions=workflow_instructions,site_wide_instructions=site_wide_instructions,site_description=site_description,directory='fold_test_temp')

        logger.info(f"Completed retro/temp fold screenshot test. Starting fold test LLM analysis for {len(fold_tests_temp)} fold tests")

        page_action_json_ranked_fold_test_temp,best_image_temp,screenshot_data_temp,resize_browser_result_temp = perform_fold_test_llm(fold_tests_temp,fold_test_paths_temp,screenshot_response_temp,resize_browser_result_temp)
        
        logger.info(f"Completed retro/temp fold test screenshot LLM analysis and the dimensions of the best image ID {best_image_temp['id']} are {best_image_temp['original_dimensions']}")


        logger.info(f"Copying screenshot from 'fold_test_temp' folder to 'temp' folder for best image for resized and non rezised for best image id {best_image['id']}")

        hlp.copy_and_rename_file(f'{best_image_temp['path']}', f'{run_rerun_path}/temp/resized_chunk_{best_image_temp['id']}.png',delete_original=False)
        hlp.copy_and_rename_file(f'{run_rerun_path}/fold_test_temp/chunk_{best_image_temp['id']}.png', f'{run_rerun_path}/temp/chunk_{best_image_temp['id']}.png',delete_original=False)

############# STOPED HERE XISOFYSFE
        previous_step_detail = hlp.extract_entities_for_evaluation(page_action_json_ranked)

        logger.info(f"Starting evaluation of the run screenshots")

        eval_json = hlp.evaluate_run_screenshots(
            workflow_instructions,
            site_wide_instructions,site_description,
            run_rerun_path,
            #[resize_screenshot_result, resize_screenshot_result_temp],
            [best_image,best_image_temp],
            previous_step_detail,
            summary,
            start_time
        )
        logger.info(f"Completed evaluation of the run screenshots")

        logger_struct.append_to_open_log_run_retry("Evaluation completed")

        actions_succeeded = eval_json['actions_succeeded']
        page_changed = eval_json['page_changed']
        overall_goal_success = eval_json['overall_goal_success']

        logger_struct.set_run_retry_value("actions_succeeded", eval_json['actions_succeeded'])
        logger_struct.set_run_retry_value("page_changed", eval_json['page_changed'])
        logger_struct.set_run_retry_value("overall_goal_success", eval_json['overall_goal_success'])

        logger_struct.set_run_retry_value("summary", eval_json['summary'])
        logger_struct.set_run_retry_value("run_advice", eval_json['run_advice'])
        logger_struct.set_run_retry_value("_expectations_analysis", eval_json['_expectations_analysis'])
        logger_struct.set_run_retry_value("_precision_analysis", eval_json['_precision_analysis'])
        logger_struct.set_run_retry_value("_error_type", eval_json['_error_type'])

        for a in eval_json['action_tasks']:
            logger_struct.set_action(a['action_id'])
            logger_struct.set_action_value(key="_error_type", value=c.get('_error_type', ''))

            for c in a.get('candidates', []):
                logger_struct.set_candidate(c['candidate_id'])
                logger_struct.set_candidate_value(key="success", value=c.get('success', ''))
                logger_struct.set_candidate_value(key="issues", value=c.get('issues', ''))
                logger_struct.set_candidate_value(key="_candidate_advice", value=c.get('_candidate_advice', ''))


        if not overall_goal_success:
            logger.info(f"Goal not achieved. Continuing to next run. Advice lenght is {len(eval_json['run_advice'])} characters and overall summary is {len(eval_json['summary'])} characters")
            # dusan_tracker.append("    -didn't succeed in goal")
            advice = f"ADVICE FROM PREVIOUS STEPS OR RUNS: Consider this advice from previous attempts to guide you what to do next.: {eval_json['run_advice']}\n"
            summary = f"OVERALL PROCESS SUMMARY: Consider this summary from previous attempts to guide you what to do next.: {eval_json['summary']}\n"

    else:
        logger.error("No response from the action API")

    # track_data["runs"].append(run_data)
    # hlp.write_track_data(track_data, folder_structures['workflow_path'])
    


    # Update run_id based on success of actions
    # If any actions succeeded and can't contune, increment run_id
    if not actions_succeeded and page_changed:  # If we cannot continue (major error)

        logger_struct.append_to_open_log_run_retry("Run needs to reset. Incremeenting rerun count.")

        rerun_ct += 1
        run_id = 0
        run_rerun_ct = 0
        logger_struct.set_rerun(rerun_ct)
        logger_struct.set_run(run_id)
        logger_struct.set_run_retry(run_rerun_ct)

        logger.info(f"PI: {sample_id}{rerun_ct}{run_id}{run_rerun_ct} Run needs to reset because actions didn't succeed and page was changed. Incrementing rerun count to {rerun_ct}. Resetting run_id to {run_id} and run_rerun_ct to {run_rerun_ct}. Navigating to initial state url metadata")

        any_actions_succeeded = False

        # dusan_tracker.append("    -Cannot continue, resetting memory, rerun count: " + str(rerun_ct) + "navigation to initial state url metadata")


        hlp.navigate_to_url_from_metadata(client_url)

        # Do NOT increment run_id here!
    elif actions_succeeded and page_changed:  # If we can continue (minor or no changes)
        

        logger_struct.append_to_open_log_run_retry("Action succeeded, page changed.")
        # dusan_tracker.append(f"    -Can continue actions_succeeded: {actions_succeeded}, page_changed: {page_changed}")
        if not overall_goal_success:  
            
            
            logger_struct.append_to_open_log_run_retry("Goal not yet achieved")
            # dusan_tracker.append(f"    -Goal not achieved but progress made, goal_achieved: {goal_achieved}, actions_succeeded: {actions_succeeded}, page_changed: {page_changed}")
            run_id += 1  # Move to the next run (correct)
            logger_struct.set_run(run_id)
            run_rerun_ct = 0
            logger_struct.set_run_retry(run_rerun_ct)
            
            logger.info(f"PI: {sample_id}{rerun_ct}{run_id}{run_rerun_ct} Progress made. Goal not yet fully achieved. Incrementing run_id to {run_id} and resetting run_rerun_ct to {run_rerun_ct}")
        else:
            
            logger.info(f"PI: {sample_id}{rerun_ct}{run_id}{run_rerun_ct} Goal achieved. Ending the workflow.")
            logger_struct.append_to_open_log_run_retry("Goal achieved")
    else: # If we can continue but the page hasn't changed
        
        logger_struct.append_to_open_log_run_retry("Page hasn't changed, but can continue at current step.")
        #dont change the run_id but increment the run rerun count
        # dusan_tracker.append(f"    -Page hasn't changed, goal_achieved: {goal_achieved}, actions_succeeded: {actions_succeeded}, page_changed: {page_changed}")
        run_rerun_ct += 1
        
        logger.info(f"PI: {sample_id}{rerun_ct}{run_id}{run_rerun_ct} Page hasn't changed, but can continue at current step. Incrementing run_rerun_ct to {run_rerun_ct}")

        logger_struct.set_run_retry(run_rerun_ct)
    

    #todo
    # why do I still get Error in main loop: Key 'summary_of_steps_so_far' already exists in this node; cannot overwrite.

2025-02-02 15:19:29,361 - Jupyter Notebook - <module> - Line: 68 - INFO - Previous run_rerun_path is  and current run_rerun_path is clients/intuit\add_non_inventory_product2\0\0\0\0
2025-02-02 15:19:29,363 - Jupyter Notebook - <module> - Line: 70 - INFO - Starting run 0 rerun 0 run_rerun 0 in clients/intuit\add_non_inventory_product2\0\0\0\0
2025-02-02 15:19:29,363 - Jupyter Notebook - <module> - Line: 85 - INFO - It's the first run for the sample. Starting fold screenshot test
2025-02-02 15:19:29,364 - Jupyter Notebook - fold_screenshot_test - Line: 7 - INFO - Requesting extract_metadata for intuit with starting width 1256 and starting height 1369
2025-02-02 15:19:30,444 - Jupyter Notebook - fold_screenshot_test - Line: 34 - INFO - Resizing browser to width 1256 and height 1256
2025-02-02 15:19:31,122 - Jupyter Notebook - fold_screenshot_test - Line: 38 - INFO - Requesting screenshot for chunk 1
2025-02-02 15:19:31,402 - Jupyter Notebook - fold_screenshot_test - Line: 47 - INFO - Comp

In [9]:
print(action_prompt)



You are part of a system that automates the exploration and documentation of web application workflows by analyzing UI states and interactions. Your role is to assist in navigating through product features by identifying and evaluating UI elements and their interactions, following standard user paths as designed in the product. Each workflow you analyze will be used to create customer support documentation, so you must prioritize generic, widely-applicable interactions that match the product's intended functionality, avoiding user-specific data or shortcuts. Avoid also asking for help through search bars for functionality questions. Search and self-help processes can be used for other things other than functional support. Your analysis should focus on actions that would be clear and repeatable in support documentation. When evaluating UI states and suggesting actions, consider how these steps would be documented for support agents and end users - they must follow standard navigation 